Table of Contents

Introduction
Part I - Probability
Part II - A/B Test
Part III - Regression

Introduction

A/B tests are very commonly performed by data analysts and data scientists. It is important that you get some practice working with the difficulties of these

For this project, you will be working to understand the results of an A/B test run by an e-commerce website. Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question. The labels for each classroom concept are provided for each question. This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria. As a final check, assure you meet all the criteria on the RUBRIC.


Part I - Probability
To get started, let's import our libraries.

In [5]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

1. Now, read in the ab_data.csv data. Store it in df

a. Read in the dataset and take a look at the top few rows here:

In [6]:
ab = pd.read_csv('ab_data.csv')
ab.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the below cell to find the number of rows in the dataset.

In [7]:
ab.shape

(294478, 5)

c. The number of unique users in the dataset.

In [8]:
ab['user_id'].nunique()

290584

d. The proportion of users converted.

In [9]:
ab['converted'].mean()

0.11965919355605512

e. The number of times the new_page and treatment don't line up.

In [10]:
new_1 = ab[(ab.group == 'control') & (ab.landing_page == 'new_page')]
new_1.count()

user_id         1928
timestamp       1928
group           1928
landing_page    1928
converted       1928
dtype: int64

In [11]:
new_2 = ab[(ab.group == 'treatment') & (ab.landing_page == 'old_page')]
new_2.count()

user_id         1965
timestamp       1965
group           1965
landing_page    1965
converted       1965
dtype: int64

In [12]:
new_1.count() + new_2.count()

user_id         3893
timestamp       3893
group           3893
landing_page    3893
converted       3893
dtype: int64

f. Do any of the rows have missing values?

In [13]:
ab.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

2. For the rows where treatment is not aligned with new_page or control is not aligned with old_page, we cannot be sure if this row truly received the new or old page. Use Quiz 2 in the classroom to provide how we should handle these rows.

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz. Store your new dataframe in df2.

In [15]:
new_1 = ab[(ab.group == 'control') & (ab.landing_page == 'old_page')]
new_2 = ab[(ab.group == 'treatment') & (ab.landing_page == 'new_page')]

In [16]:
#Concat 
df2 = pd.concat([new_1,new_2], keys=['control','treatment'], axis = 0)
df2.reset_index(drop=True)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,864975,2017-01-21 01:52:26.210827,control,old_page,1
3,936923,2017-01-10 15:20:49.083499,control,old_page,0
4,719014,2017-01-17 01:48:29.539573,control,old_page,0
5,644214,2017-01-22 02:05:21.719434,control,old_page,1
6,847721,2017-01-17 14:01:00.090575,control,old_page,0
7,650559,2017-01-24 11:55:51.084801,control,old_page,0
8,935734,2017-01-17 20:33:37.428378,control,old_page,0
9,746742,2017-01-23 11:38:29.592148,control,old_page,0


In [17]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

3. Use df2 and the cells below to answer questions for Quiz3 in the classroom.

a. How many unique user_ids are in df2?

In [18]:
df2['user_id'].nunique()

290584

b. There is one user_id repeated in df2. What is it?

source: https://stackoverflow.com/questions/14657241/how-do-i-get-a-list-of-all-the-duplicate-items-using-pandas-in-python

c. What is the row information for the repeat user_id?

In [19]:
df2[df2.duplicated(subset = ['user_id'], keep=False)]

user_id                   timestamp      group landing_page  \
treatment 1899   773192  2017-01-09 05:37:58.781806  treatment     new_page   
          2893   773192  2017-01-14 02:55:59.590927  treatment     new_page   

                converted  
treatment 1899          0  
          2893          0

d. Remove one of the rows with a duplicate user_id, but keep your dataframe as df2.

In [20]:
df2 = df2.drop_duplicates(subset = ['user_id'],keep = 'last')
df2

user_id                   timestamp      group landing_page  \
control   0        851104  2017-01-21 22:11:48.556739    control     old_page   
          1        804228  2017-01-12 08:01:45.159739    control     old_page   
          4        864975  2017-01-21 01:52:26.210827    control     old_page   
          5        936923  2017-01-10 15:20:49.083499    control     old_page   
          7        719014  2017-01-17 01:48:29.539573    control     old_page   
          15       644214  2017-01-22 02:05:21.719434    control     old_page   
          16       847721  2017-01-17 14:01:00.090575    control     old_page   
          18       650559  2017-01-24 11:55:51.084801    control     old_page   
          19       935734  2017-01-17 20:33:37.428378    control     old_page   
          25       746742  2017-01-23 11:38:29.592148    control     old_page   
          28       913579  2017-01-24 09:11:39.164256    control     old_page   
          30       690284  2017-01-13 17:22:57.182769    control     old_page   
          34       710349  2017-01-11 22:24:44.226492    control     old_page   
          35       677533  2017-01-23 17:48:50.491821    control     old_page   
          36       831737  2017-01-11 21:18:20.911015    control     old_page   
          40       771087  2017-01-16 00:05:29.983919    control     old_page   
          42       896163  2017-01-22 09:10:20.753218    control     old_page   
          43       862225  2017-01-08 14:49:37.335432    control     old_page   
          44       939593  2017-01-05 09:15:31.984283    control     old_page   
          45       702260  2017-01-18 13:55:31.488221    control     old_page   
          50       670941  2017-01-05 08:16:41.306478    control     old_page   
          51       850231  2017-01-18 17:18:04.790584    control     old_page   
          55       685794  2017-01-20 14:54:58.150621    control     old_page   
          57       714733  2017-01-03 08:22:37.904146    control     old_page   
          58       710967  2017-01-10 02:19:22.842142    control     old_page   
          59       680201  2017-01-11 10:38:45.952887    control     old_page   
          60       790863  2017-01-19 11:02:39.220320    control     old_page   
          61       717595  2017-01-23 18:19:08.148166    control     old_page   
          62       779854  2017-01-11 21:28:30.735359    control     old_page   
          63       916307  2017-01-19 17:27:38.676600    control     old_page   
...                   ...                         ...        ...          ...   
treatment 294417   924332  2017-01-15 19:38:52.858024  treatment     new_page   
          294422   849625  2017-01-06 17:54:07.563311  treatment     new_page   
          294424   929723  2017-01-10 15:13:48.352399  treatment     new_page   
          294427   774769  2017-01-03 06:01:36.251836  treatment     new_page   
          294430   733871  2017-01-21 17:54:08.810964  treatment     new_page   
          294432   844588  2017-01-16 20:48:19.567178  treatment     new_page   
          294433   641244  2017-01-07 16:57:26.193171  treatment     new_page   
          294434   676072  2017-01-14 17:26:02.495442  treatment     new_page   
          294435   886374  2017-01-07 13:43:39.202634  treatment     new_page   
          294437   676732  2017-01-03 23:06:45.459467  treatment     new_page   
          294439   862218  2017-01-04 10:43:07.846494  treatment     new_page   
          294441   798826  2017-01-23 16:50:13.788528  treatment     new_page   
          294442   836721  2017-01-12 17:37:50.966955  treatment     new_page   
          294444   844901  2017-01-15 17:46:36.622726  treatment     new_page   
          294445   653124  2017-01-14 13:44:51.745491  treatment     new_page   
          294446   909437  2017-01-18 14:49:49.064452  treatment     new_page   
          294448   776137  2017-01-12 05:53:12.386730  treatment     new_page   
          294449   883344  2017-01-22 23:15:58.64

4. Use df2 in the below cells to answer the quiz questions related to Quiz 4 in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [21]:
df2['converted'].mean()

0.11959708724499628

b. Given that an individual was in the control group, what is the probability they converted?

souce: https://stackoverflow.com/questions/36684013/extract-column-value-based-on-another-column-pandas-dataframe

In [22]:
df2[df2['group'] == 'control']['converted'].mean()

0.1203863045004612

c. Given that an individual was in the treatment group, what is the probability they converted?

In [23]:
df2[df2['group'] == 'treatment']['converted'].mean()

0.11880806551510564

d. What is the probability that an individual received the new page?

In [24]:
df2.shape

(290584, 5)

In [25]:
len(df2.query("landing_page == 'new_page'"))

145310

In [27]:
145311/float(290585)

0.5000636646764286

e. Use the results in the previous two portions of this question to suggest if you think there is evidence that one page leads to more conversions? Write your response below.

While comparing treatment and control, it turns out that control scored higher than treatment. I guess it is hard to tell that one page leads to more conversion because the probability that the individual received the page is 0.5.


Part II - A/B Test
Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time? How long do you run to render a decision that neither page is better than another?

These questions are the difficult parts associated with A/B tests in general.

1. For now, consider you need to make the decision just based on all the data provided. If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be? You can state your hypothesis in terms of words or in terms of $p_{old}$ and $p_{new}$, which are the converted rates for the old and new pages.

source: https://matplotlib.org/users/mathtext.html

Ho : Pold > Pnew

Ho: Pold = Pnew

H1 : Pold < Pnew

2. Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the converted success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the converted rate in ab_data.csv regardless of the page.


Use a sample size for each page equal to the ones in ab_data.csv.


Perform the sampling distribution for the difference in converted between the two pages over 10,000 iterations of calculating an estimate from the null.


Use the cells below to provide the necessary parts of this simulation. If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem. You can use Quiz 5 in the classroom to make sure you are on the right track.


a. What is the convert rate for $p_{new}$ under the null?

In [28]:
Pnew = df2.converted.mean()
Pnew

0.11959708724499628

In [29]:
Pold = df2.converted.mean()
Pold

0.11959708724499628

In [30]:
nnew = df2.query('landing_page == "new_page"').shape[0] 
nnew

145310

In [31]:
nold = df2.query('landing_page == "old_page"').shape[0] #147239
nold

145274

In [32]:
new_page_converted = np.random.binomial(1, Pnew, nnew)
new_page_converted

array([0, 0, 0, ..., 0, 0, 1])

In [33]:
npc = new_page_converted.mean()
npc

0.11906269355171702

In [34]:
old_page_converted = np.random.binomial(1, Pold, nold)
old_page_converted

array([1, 0, 0, ..., 0, 0, 0])

In [35]:
opc = old_page_converted.mean()
opc

0.1197599019783306

In [36]:
#simulated diff
npc - opc

-0.00069720842661358251